# El Yapımı Standardizer

🧑🏻‍🏫 Bu challenge’da *kendi* StandardScaler’ımızı oluşturacağız. Neden mi? Güzel soru!

🎯 Bu egzersizin amaçları şunlardır:
- `stateless transformers` ile `stateful transformers` arasındaki farkı anlamak
- `FeatureUnion` ile çalışmak

## (1) 📚 Stateless Transformer vs. Stateful Transformer

🔢 Aşağıdaki training set’i ve aşağıdaki test set’i göz önünde bulundurun...

In [1]:
import numpy as np
import pandas as pd

X_train = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 4, 1: 5, 2: 6},
    'C': {0: 7, 1: 8, 2: 9}
})

print("This is the training dataset:")
display(X_train)

X_test = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 10}
})

print("This is the test dataset:")
display(X_test)

This is the training dataset:


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


This is the test dataset:


,A,B,C
0,1,2,3
1,2,3,4
2,3,4,10


🛠 ...ve aşağıdaki FeatureUnion’ı düşünün; bu yapı:
- feature’ları ölçekler
- diğer (ölçeklenmemiş) feature’ların ortalaması olan yeni bir feature oluşturur

In [2]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, make_union

standard_scaler = StandardScaler()
feature_averager = FunctionTransformer(lambda df: pd.DataFrame(1/3 * (df["A"] + df["B"] + df["C"])))

pipeline = make_union(standard_scaler, feature_averager)
pipeline

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x7f0a3c130c20>))])

Hadi:
- pipeline’ı training set üzerinde fit edelim
- hem training set’i hem de test set’i transform edelim

In [3]:
pipeline.fit(X_train)

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x7f0a3c130c20>))])

In [4]:
X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_train_transformed

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,4.0
1,0.000000,0.000000,0.000000,5.0
2,1.224745,1.224745,1.224745,6.0


In [5]:
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))
X_test_transformed

,0,1,2,3
0,-1.224745,-3.674235,-6.123724,2.000000
1,0.000000,-2.449490,-4.898979,3.000000
2,1.224745,-1.224745,2.449490,5.666667


👨🏻‍🏫 `StandardScaler` ile `FunctionTransformer`’ın temelde ne kadar farklı olduğuna dikkat edin ❗️

Pipeline’ı fit edip hem training set’i hem de test set’i transform ettiğimizde:

* **`FunctionTransformer (feature_averager)`**:
    * *.fit()* sırasında hiçbir şey “öğrenmedi”
    * yalnızca **stateless bir dönüşüm** uyguladı: $ \large (X_1, X_2, X_3) \rightarrow \frac{(X_1 + X_2 + X_3)}{3}$


* **`StandardScaler`**:
    * *.fit()* sırasında $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini “öğrendi”
    * bu öğrenilen değerleri kullanarak hem training set’te hem de test set’te **stateful bir dönüşüm** gerçekleştirdi:
        * $ \large X_{\color{blue}{train-scaled}} =  \frac{X_{\color{blue}{train}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$
        * $ \large X_{\color{red}{test-scaled}} =  \frac{X_{\color{red}{test}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$

## (2) 💻 Kendi stateful transformer’ınızı oluşturun

🤔 Peki ya kendi **stateful custom transformer**’ımızı kodlamak istersek?

💪 Kendi class’ımızı yazabiliriz!

# El Yapımı Standardizer

### (2.1) 💻 Custom Standardizer

❓ **Sorular: Kendi class’ınızı kodlama** ❓

1. Kendi `CustomStandardizer` class’ınızı yazın  
    * Scikit Learn’deki `StandardScaler` ile **birebir aynı davranışı** göstermelidir. Yani:
        * $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini hesaplayan (“öğrenen”) bir `.fit()` metodu
        * ve bir `.transform()` metodu olmalıdır.

2. Bu class’ı `X_train` üzerinde fit edin

3. Hem `X_train` hem de `X_test` üzerinde transform uygulayın

4. Her şeyi doğru yaptığınızdan emin olmak için `CustomStandardizer`’ınızı Scikit Learn’deki `StandardScaler` ile karşılaştırın!

In [15]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator

class CustomStandardizer(TransformerMixin, BaseEstimator):

    def __init__(self):
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        self.mean_ = X.mean()
        self.std_ = X.std(ddof=0)
        return self

    def transform(self, X, y=None):
        return (X - self.mean_) / self.std_

    def inverse_transform(self, X):
        return (X * self.std_) + self.mean_

In [16]:
#########################################
# 2 - CustomStandardizer Class'ını Eğit (Fit) #
#########################################

custom_standardizer = CustomStandardizer()
custom_standardizer.fit(X_train)

CustomStandardizer()

In [17]:
X_train_custom_scaled = custom_standardizer.transform(X_train)
X_test_custom_scaled = custom_standardizer.transform(X_test)

X_train_custom_scaled, X_test_custom_scaled

(          A         B         C
 0 -1.224745 -1.224745 -1.224745
 1  0.000000  0.000000  0.000000
 2  1.224745  1.224745  1.224745,
           A         B         C
 0 -1.224745 -3.674235 -6.123724
 1  0.000000 -2.449490 -4.898979
 2  1.224745 -1.224745  2.449490)

🧪 **Kodunu Test Et**

In [18]:
from nbresult import ChallengeResult

tmp = CustomStandardizer()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/selen/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/selen/data/S17D1-S-Data-hand-made-standardizer/tests
plugins: anyio-4.8.0, typeguard-4.4.2, dash-3.3.0
collecting ... collected 1 item

test_standardizer.py::TestStandardizer::test_solution PASSED             [100%]

============================== 1 passed in 0.15s ===============================


💯 You can commit your code:

git add tests/standardizer.pickle

git commit -m 'Completed standardizer step'

git push origin master



<details>
<summary>💡 <i>İpuçları</i> (🧪 eğer yukarıdaki testler yalnızca küçük bir farkla başarısız oluyorsa)</summary>

* `np.std()` ile `pd.std()` metotları arasında **küçük bir fark** olduğuna dikkat edin!
    
* Bu [Stack Overflow gönderisi](https://stackoverflow.com/questions/44220290/sklearn-standardscaler-result-different-to-manual-result) yardımcı olabilir 😉
      
</details>

<details>
<summary>💡 <i>Çözüm</i></summary>

Eğer `pd.std` metodunu kullanıyorsanız, scikit-learn’ün `StandardScaler` davranışını birebir taklit edebilmek için `ddof=0` parametresini de kullanmanız gerekir.      
</details>

### (2.2) 💻 Inverse Transform

❓ **Question (Inverse Transform)** ❓

_StandardScaler_ from Scikit Learn has a [`.inverse_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) method that helps you revert back to the unscaled dataset.

1. Go back to your `CustomStandardizer` class and implement your own `.inverse_transform()` method.

2. Try it on your scaled training set and your scaled test set.

In [19]:
    def inverse_transform(self, X):
        """
        Scaled veriyi orijinal ölçeğine geri döndürür
        """
        X_original = (X * self.std_) + self.mean_
        return X_original

In [20]:
# Inverse transform’unuzu deneyin ve doğru kodladığınızdan emin olmak için aşağıdaki testi çalıştırın
X_train_inverse_transformed = custom_standardizer.inverse_transform(X_train_custom_scaled)
X_test_inverse_transformed = custom_standardizer.inverse_transform(X_test_custom_scaled)

X_train_inverse_transformed, X_test_inverse_transformed

(     A    B    C
 0  1.0  4.0  7.0
 1  2.0  5.0  8.0
 2  3.0  6.0  9.0,
      A    B     C
 0  1.0  2.0   3.0
 1  2.0  3.0   4.0
 2  3.0  4.0  10.0)

🧪 **Kodunu Test Et**

In [21]:
assert np.allclose(X_train_inverse_transformed, X_train)
assert np.allclose(X_test_inverse_transformed, X_test)

### (2.3) 💻 Özel (custom) pipeline tamamlandı!

💪 Scikit-Learn’in `StandardScaler`’ını birebir kopyalamayı başardık.

🌶 Şimdi biraz daha heyecan katalım!

❓ **Soru: Önceki `CustomStandardizer` özel transformer’ını bir shrink factor ile geliştirin** ❓

`CustomStandardizer(shrink_factor = 1)` class’ı, ölçeklemeyi daha güçlü hale getirmek için **ek bir argüman** almalıdır. Bu durumda ölçekleme, $\sigma_{\color{blue}{train}}$ ile **orantılı** olacaktır 👇:

- $ \large X_{\color{blue}{train-scaled}} =  \left(\frac{X_{\color{blue}{train}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$
- $ \large X_{\color{red}{test-scaled}} =  \left(\frac{X_{\color{red}{test}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$

In [23]:
###################################
# Custom Standardizer             #
###################################



import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator

class CustomStandardizer(TransformerMixin, BaseEstimator):

    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        self.mean_ = X.mean()
        self.std_ = X.std(ddof=0)
        return self

    def transform(self, X, y=None):
        X_scaled = (X - self.mean_) / self.std_
        return X_scaled * (1 / self.shrink_factor)

    def inverse_transform(self, X, y=None):
        X_unscaled = X * self.shrink_factor
        return (X_unscaled * self.std_) + self.mean_

🧪 **Yeni `CustomStandardizer` özel transformer’ınızı test edin** (`shrink_factor = 2` ile):

- `X_train` üzerinde fit edin
- Hem `X_train` hem de `X_test` üzerinde transform uygulayın
- Dönüştürülmüş DataFrame’leri `X_train_transformed` ve `X_test_transformed` değişkenlerinde saklayın

# El Yapımı Standardizer

In [24]:
custom_standardizer_sf = CustomStandardizer(shrink_factor=2)
custom_standardizer_sf.fit(X_train)

X_train_transformed = custom_standardizer_sf.transform(X_train)
X_test_transformed = custom_standardizer_sf.transform(X_test)

X_train_transformed, X_test_transformed

(          A         B         C
 0 -0.612372 -0.612372 -0.612372
 1  0.000000  0.000000  0.000000
 2  0.612372  0.612372  0.612372,
           A         B         C
 0 -0.612372 -1.837117 -3.061862
 1  0.000000 -1.224745 -2.449490
 2  0.612372 -0.612372  1.224745)

🧪 **Kodunu Test Et**

In [25]:
from nbresult import ChallengeResult

tmp = CustomStandardizer(shrink_factor=2).fit(X_train)
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'new_standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/selen/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/selen/data/S17D1-S-Data-hand-made-standardizer/tests
plugins: anyio-4.8.0, typeguard-4.4.2, dash-3.3.0
collecting ... collected 1 item

test_new_standardizer.py::TestNewStandardizer::test_solution PASSED      [100%]

============================== 1 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/new_standardizer.pickle

git commit -m 'Completed new_standardizer step'

git push origin master



In [26]:
# Aşağıdaki hücreleri çalıştırarak doğru dönüşümleri elde ettiğinizden emin olun
truth_train = np.array([
    [-0.612372, -0.612372, -0.612372],
    [0.000000, 0.000000, 0.000000],
    [0.612372, 0.612372, 0.612372]
])
truth_test = np.array([
    [-0.612372, -1.837117, -3.061862],
    [ 0.        , -1.224745, -2.449490],
    [ 0.612372, -0.612372,  1.224745]])

In [27]:
# # Assert’ler
np.allclose(X_train_transformed, truth_train)

True

In [28]:
# # Assert - Test
np.allclose(X_test_transformed, truth_test)

True

❓ **Soru: Önceki `FeatureAverager` özel transformer’ını “tweak” edin** ❓

Bu değiştirilmiş `FeatureAverager()` class’ı:
- hâlâ üç farklı feature’ın ortalamasını hesaplar...
- ...ve ardından sonucu her satır için maksimum değere böler

_Not_: Bu işlemi yorumlamaya çalışmayın. Gerçek bir anlamı yok; amacımız yalnızca özel bir class kodlama becerilerimizi pratik etmek.

$$(X_1, X_2, X_3) \rightarrow \frac{mean(X_1 + X_2 + X_3)}{max(X_1, X_2, X_3)}$$

_Not_: Burada **stateful** mı yoksa **stateless** bir dönüşümle mi uğraştığımızı dikkatlice düşünün (yani, *stateful* `StandardScaler`’da yaptığımız gibi dönüşümden önce tüm training verisi hakkında bilgi öğrenmemiz gerekiyor mu, yoksa yukarıdaki *stateless* `feature_averager` gibi mi çalışıyor?). İlki söz konusuysa `.fit()` için bir şeyler yazmamız gerekir; değilse gerekmez!

In [29]:
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator

class FeatureAverager(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # Stateless transformer → öğrenilecek bir şey yok
        return self

    def transform(self, X, y=None):
        # Satır bazında ortalama
        row_mean = X.mean(axis=1)

        # Satır bazında maksimum
        row_max = X.max(axis=1)

        # Ortalama / maksimum
        transformed = row_mean / row_max

        # sklearn uyumu için 2D array döndür
        return transformed.values.reshape(-1, 1)

🧪 **Test you `FeatureAverager` custom transformer** by fitting on `X_train`  and transforming both `X_train` and `X_test`

In [30]:
tmp = FeatureAverager()

tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

In [31]:
from nbresult import ChallengeResult

tmp = FeatureAverager()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'feature_averager',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/selen/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/selen/data/S17D1-S-Data-hand-made-standardizer/tests
plugins: anyio-4.8.0, typeguard-4.4.2, dash-3.3.0
collecting ... collected 1 item

test_feature_averager.py::TestFeatureAverager::test_solution PASSED      [100%]

============================== 1 passed in 0.14s ===============================


💯 You can commit your code:

git add tests/feature_averager.pickle

git commit -m 'Completed feature_averager step'

git push origin master



❓ **Soru (Feature Union)** ❓

1. Hem `CustomStandardizer` hem de `FeatureAverager` kullanarak bir `FeatureUnion` pipeline oluşturun ve bunu `pipeline` adlı bir değişkende saklayın.

2. Pipeline’ı `X_train` üzerinde fit edin ve hem `X_train` hem de `X_test` üzerinde transform uygulayın (`shrink_factor = 3`).

3. Bu challenge’ın son testini geçtiğinizden emin olun.

In [32]:
#####################
# 1 - Feature Union #
#####################

from sklearn.pipeline import FeatureUnion

pipeline = FeatureUnion([
    ("standardizer", CustomStandardizer(shrink_factor=3)),
    ("feature_averager", FeatureAverager())
])

In [33]:
#########################
# 2 - Fit ve Transform #
#########################

pipeline.fit(X_train)

FeatureUnion(transformer_list=[('standardizer',
                                CustomStandardizer(shrink_factor=3)),
                               ('feature_averager', FeatureAverager())])

In [34]:
from nbresult import ChallengeResult

tmp = pipeline
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))



result = ChallengeResult(
    'feature_union_custom_transformers',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/selen/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/selen/data/S17D1-S-Data-hand-made-standardizer/tests
plugins: anyio-4.8.0, typeguard-4.4.2, dash-3.3.0
collecting ... collected 1 item

test_feature_union_custom_transformers.py::TestFeatureUnionCustomTransformers::test_solution PASSED [100%]

============================== 1 passed in 0.15s ===============================


💯 You can commit your code:

git add tests/feature_union_custom_transformers.pickle

git commit -m 'Completed feature_union_custom_transformers step'

git push origin master



🏁 Tebrikler! Kendi Transformer’ınızı nasıl oluşturacağınızı keşfettiniz!

💾 Notebook’unuzu `git add / commit / push` yapmayı unutmayın...

🚀 ... ve bir sonraki challenge’a geçin!